# Podman (Containerisation)

## Goals
- Understand Podman’s core principles: **daemonless containers**, **OCI compatibility**, and **rootless-by-default** workflows.
- Know what Podman is used for and where it fits compared to Docker.
- Learn the most important commands.
- Understand how Podman networking and volumes work (including rootless differences).
- Understand how Compose-style workflows work with Podman.


## Prerequisites
- Basic Linux concepts help: processes, filesystems, permissions.
- Basic networking: IPs, ports, DNS.

Notes:
- Podman is primarily a Linux-native tool.
- Rootless containers behave differently from rootful containers in networking and filesystem permissions.


## What is Podman?

**Podman** is a container engine focused on running OCI containers **without a central daemon**.

Key ideas:
- **Daemonless**: the CLI talks directly to a library/runtime to create containers (no always-running `dockerd`).
- **Rootless**: you can run containers as an unprivileged user (improves security and reduces blast radius).
- **OCI-aligned**: works with OCI images and OCI runtimes (commonly `crun` or `runc`).
- **Pods**: a first-class concept (like Kubernetes pods) where multiple containers can share namespaces (especially networking).


## Podman vs Docker (practical differences)
- **Daemon**:
  - Docker typically uses a daemon (`dockerd`).
  - Podman is daemonless for normal use.
- **Rootless**:
  - Docker can be rootless, but many setups run Docker with elevated privileges.
  - Podman commonly defaults to rootless.
- **Pods**:
  - Podman supports pods natively (useful for grouping containers that should share networking).
- **Compatibility**:
  - Many `docker ...` CLI patterns map closely to `podman ...`.
  - If you need a Docker-compatible API socket, Podman can provide one via `podman system service`.


## Core concepts (minimum you should know)
- **Image**: OCI image layers + metadata.
- **Container**: a running/stopped instance of an image.
- **Pod**: group of containers sharing a network namespace (and optionally others).
- **Network**: how containers/pods connect (bridge/host/none; implementation differs rootless vs rootful).
- **Volume / bind mount**: persistence outside the container’s writable layer.
- **Kubernetes YAML interop**: Podman can generate or consume K8s-like YAML in some workflows.


## Most important commands (cheat sheet)

Images:
```bash
podman pull nginx:1.25
podman images
podman build -t myapp:dev .
podman push myapp:dev docker://myrepo/myapp:dev
```

Containers:
```bash
podman ps
podman ps -a
podman run --rm -it ubuntu:24.04 bash
podman run -d --name web -p 8080:80 nginx:1.25
podman logs -f web
podman exec -it web sh
podman stop web
podman rm web
```

Pods:
```bash
podman pod create --name apppod -p 8080:8080
podman run -d --pod apppod --name web myapp:dev
podman run -d --pod apppod --name worker myworker:dev
podman pod ps
podman pod stop apppod
podman pod rm apppod
```

Networking + storage:
```bash
podman network ls
podman network inspect <network>
podman volume ls
podman volume inspect <volume>
```


## Networking: how container IPs work (rootful vs rootless)

### Rootful Podman (typical server/admin setup)
- Containers run with full OS primitives available.
- Podman creates bridge networks and assigns container IPs (similar mental model to Docker).
- A DNS helper may provide container-name resolution on user-defined networks (implementation varies by Podman version/setup).

### Rootless Podman (common developer setup)
- Rootless networking cannot program the host kernel networking stack as freely.
- Podman commonly uses user-mode networking (e.g., `slirp4netns` or `pasta`).
- Port publishing still works, but is implemented via user-space forwarding.

Practical implications:
- **Performance**: rootless user-mode networking can be slower than rootful bridge networking.
- **Inbound reachability**: published ports work; direct access to container IPs from other hosts is usually not the model.
- **Pods**: containers in the same pod share a network namespace, so they can talk over `localhost`.

Debugging tips:
```bash
podman inspect web | jq '.[0].NetworkSettings'
podman port web
podman network inspect <network>
```


## Volumes and mounts (how persistence works)

Same core idea as Docker: the container’s writable layer is disposable; persist data using volumes/mounts.

### Named volumes
- Managed by Podman.
- Rootless volumes are typically stored under your home directory (e.g., `~/.local/share/containers/storage/volumes/`).

Example:
```bash
podman volume create pgdata
podman run -d --name db \
  -e POSTGRES_PASSWORD=dev \
  -v pgdata:/var/lib/postgresql/data \
  postgres:16
```

### Bind mounts
- Mount a specific host path into the container.
- On SELinux systems you may need relabel flags (common patterns):
  - `:Z` private label (only that container)
  - `:z` shared label (multiple containers)

Example:
```bash
podman run --rm -it \
  -v "$PWD":/app:Z \
  -w /app \
  python:3.12-slim python -m http.server 8000
```

Common pitfall:
- Rootless containers often run with user-namespace mapping, so file ownership/permissions can surprise you.


## Compose-style workflows with Podman

Podman can run Compose files in a few ways (depends on your installation/version):
- `podman compose ...` (Compose plugin)
- `podman-compose ...` (separate tool)

The goal is compatibility with the **Compose Specification**:
- Define services, networks, and volumes.
- Bring the stack up/down as a unit.

Conceptually it works like Docker Compose:
1) Build/pull images.
2) Create networks and volumes.
3) Create containers and attach them to networks.
4) Start containers.

Example commands:
```bash
podman compose up -d --build
podman compose logs -f
podman compose down
```


## What Podman is capable of
- Run OCI containers rootless (strong default security posture).
- Manage pods (shared networking) and model Kubernetes-like grouping locally.
- Integrate with systemd for long-running services.

Example: generate a systemd unit (pseudo-workflow)
```bash
# PSEUDO-CODE
podman run -d --name web -p 8080:80 nginx:1.25
podman generate systemd --name web --files --new
# then move unit file to ~/.config/systemd/user/ and enable it
systemctl --user daemon-reload
systemctl --user enable --now container-web.service
```

Kubernetes YAML interop (conceptual):
```bash
# PSEUDO-CODE
podman generate kube web > web.yaml
podman play kube web.yaml
```


## Pitfalls & quick tips
- Rootless networking/permissions are different; test your assumptions.
- Prefer stable names and labels to avoid accidental replacements.
- Treat container state/config as code: version control your compose files and pod definitions.
- Be explicit about mounts and SELinux relabeling on enforcing systems.

## Exercises
- Run a small web container rootless and publish a port.
- Create a pod with two containers and verify they can talk via `localhost`.
- Run a Compose stack using `podman compose` (or `podman-compose`) and inspect the networks/volumes created.

## References
- Podman Docs: https://podman.io/docs
- Compose Spec: https://compose-spec.io/
